In [1]:
import imagej
import scyjava
import os
import shutil
from datetime import datetime
 
scyjava.config.add_option('-Xmx6g')
#use 'sc.fiji:fiji' otherwise fiji won't be initialized, you can also connect it to a local application (might be better
# in case you have important plugins installed)

ij = imagej.init('C:/fiji-win64/Fiji.app', mode = 'interactive')

def move_images(src_folder, dest_folder, image_extensions=[".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp"]):
            """
            Move all images from the source folder to the destination folder.

            Parameters:
            - src_folder: The source directory containing the images.
            - dest_folder: The destination directory to move the images to.
            - image_extensions: List of file extensions to consider as images. Default includes common image formats.
            """

    
            # Ensure destination folder exists, if not, create it
            if not os.path.exists(dest_folder):
                os.makedirs(dest_folder)
    
            # Loop through each file in the source folder
            for filename in os.listdir(src_folder):
                # Check if the file has one of the image extensions
                if any(filename.endswith(ext) for ext in image_extensions):
                    src_path = os.path.join(src_folder, filename)
                    dest_path = os.path.join(dest_folder, filename)
            
                # Move the file
                    shutil.move(src_path, dest_path)
                    
                    
 
# Define the directory containing your images
image_directory = 'C:/Users/lsest/OneDrive - Danmarks Tekniske Universitet/Experiments/V28 Online image Germination/Sorted_CNN/Part2'
image_files = os.listdir(image_directory)

# Define folders for Csv file
source_folder = 'C:/Users/lsest/Desktop/pyimageJ_results'  # Replace with your source folder path
target_folder = 'C:/Users/lsest/Desktop/pyimageJ_results'  # Replace with your target folder path

#Give a folder for macro input 
destination_image_path = "C:/Users/lsest/Desktop/pyimageJ_inputs/img.png"
 
# Define your macro
macro = """
// Process the image
open("C:/Users/lsest/Desktop/pyimageJ_inputs/img.png")
close("dummy")

//Duplicate the original image 
selectWindow("img.png");
run("Duplicate...", "title=original");
selectWindow("img.png");
run("8-bit");
run("Invert");

run("Auto Threshold", "method=Otsu white");

run("Fill Holes");
run("Erode");
run("Watershed");

// Get rid of aggregates
run("Duplicate...", "title=img-1.png");
run("Analyze Particles...", "size=3000-Infinity show=Masks");
run("Invert");
imageCalculator("Subtract create", "Mask of img-1.png","img.png");
selectImage("Result of Mask of img-1.png");
run("Clear Results");
roiManager("reset");
 
//Analyse single cells
run("Invert");
run("Analyze Particles...", "size=200-Infinity circularity=0.50-1.00 show=Nothing display add exclude");
run("Clear Results");

n = roiManager("count");

run("Set Measurements...", "area mean standard modal min centroid center perimeter bounding fit shape feret's integrated median skewness kurtosis area_fraction redirect=None decimal=3");



// Measure ROIs
for (i=0; i<n; i++){
	selectWindow("original");
	roiManager("Select", i); 
	run("Measure");
}


saveAs("Results",'C:/Users/lsest/Desktop/pyimageJ_results/Measurements.csv');
roiManager("reset");
run("Close All");
run("Clear Results");
"""


#Analyse all images in the file location
for image_file in image_files: 
    if image_file.endswith(('.tif' , '.jpg' , '.png')):
        
        image_path = os.path.join(image_directory, image_file)
        
        # Get the last modification time
        modification_time = os.path.getmtime(image_path)

        # Convert it to a more readable format
        readable_time = datetime.fromtimestamp(modification_time).strftime('%Y-%m-%d %H:%M:%S')
        shutil.copy(image_path, destination_image_path)
        
        
        
    
        ij.py.run_macro('newImage("dummy", "8-bit", 1, 1, 1);')
        
        ij.py.run_macro(macro)
        
        # Rename and move the output CSV file
        original_csv = os.path.join(source_folder, 'Measurements.csv')
        if os.path.exists(original_csv):
            # Get modification time of the image and format it
            modification_time = os.path.getmtime(image_path)
            readable_time = datetime.fromtimestamp(modification_time).strftime('%Y!%m!%d %H-%M-%S')

            # Generate new file name
            new_csv_name = f'{readable_time}.csv'
            new_csv_path = os.path.join(target_folder, new_csv_name)

            # Rename and move the file
            shutil.move(original_csv, new_csv_path)
            print(f'Renamed and moved CSV to {new_csv_path}')
        
# Example usage
source_folder = 'C:/Users/lsest/Desktop/pyimageJ_results'  # Replace with your source folder path
target_folder = 'C:/Users/lsest/Desktop/pyimageJ_results'  # Replace with your target folder path

 
# Shutdown ImageJ
ij.getContext().dispose()

Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!02!29 09-37-28.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!02!29 10-37-30.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!02!29 11-37-32.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!02!29 12-37-32.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!02!29 13-37-34.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!02!29 14-37-36.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!03!01 09-33-10.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!03!01 10-33-08.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!03!01 11-33-10.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!03!01 12-33-12.csv
Renamed and moved CSV to C:/Users/lsest/Desktop/pyimageJ_results\2024!03!01 13-33-14.csv


## Analyse results

In [2]:
import pandas as pd 
import os
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# Set the working directory to where the model is located
model_dir = 'C:/Users/lsest/OneDrive - Danmarks Tekniske Universitet/Experiments/V26 Online image analysis/Datasets/Ocelloscope/500_trainingset/Measurements'
os.chdir(model_dir)

# Create a new model instance
loaded_model = xgb.XGBClassifier()

# Load the model from the file
loaded_model.load_model('V26_Classifiermodel.json')

# List to store all class percentages
class_percentages_list = []

# Directory where the CSV files are located
csv_files_dir = 'C:/Users/lsest/Desktop/pyimageJ_results'


# Process each CSV file in the directory
for file_name in os.listdir(csv_files_dir):
    if file_name.endswith('.csv'):
        csv_file_path = os.path.join(csv_files_dir, file_name)
        
        # Read the CSV file
        X_test = pd.read_csv(csv_file_path)

        # Predict outcomes using the loaded model
        y_pred = loaded_model.predict(X_test)

        # Calculate class percentages
        unique, counts = np.unique(y_pred, return_counts=True)
        percentages = dict(zip(unique, counts * 100 / len(y_pred)))

        # Print class percentages
        print(f"Class percentages for {file_name}: {percentages}")

        # Add the file name and percentages to the list
        percentages['file_name'] = file_name
        class_percentages_list.append(percentages)

# Create a DataFrame from the list
class_percentages_df = pd.DataFrame(class_percentages_list)

# Set file_name as the first column
cols = ['file_name'] + [col for col in class_percentages_df if col != 'file_name']
class_percentages_df = class_percentages_df[cols]

# Save class percentages to a single CSV file
output_file = 'C:/Users/lsest/Desktop/xgb_results/class_percentagesmiss.csv'
class_percentages_df.to_csv(output_file, index=False)

        



Class percentages for 2024!02!29 09-37-28.csv: {0: 5.527123848515865, 1: 16.274309109518935, 2: 10.440122824974411, 3: 14.73899692937564, 4: 53.01944728761515}
Class percentages for 2024!02!29 10-37-30.csv: {0: 6.541019955654102, 1: 15.964523281596453, 2: 10.310421286031042, 3: 12.971175166297117, 4: 54.212860310421284}
Class percentages for 2024!02!29 11-37-32.csv: {0: 4.119850187265918, 1: 14.325842696629213, 2: 9.737827715355806, 3: 12.827715355805243, 4: 58.98876404494382}
Class percentages for 2024!02!29 12-37-32.csv: {0: 4.433962264150943, 1: 14.81132075471698, 2: 10.471698113207546, 3: 13.018867924528301, 4: 57.264150943396224}
Class percentages for 2024!02!29 13-37-34.csv: {0: 5.622489959839357, 1: 16.566265060240966, 2: 11.144578313253012, 3: 12.650602409638553, 4: 54.01606425702811}
Class percentages for 2024!02!29 14-37-36.csv: {0: 5.882352941176471, 1: 17.3374613003096, 2: 11.764705882352942, 3: 12.074303405572756, 4: 52.94117647058823}
Class percentages for 2024!03!01 09-3